# Exemplary SA-CCR allocation under consideration of a minimum transfer amount.

In [1]:
import QuantLib as ql
from IPython.core.display import display, Markdown
from scipy import optimize
from allocation.eulerAllocator import EulerAllocator
from collateralAgreement.collateralAgreement import CollateralAgreement
from instruments.interestRateInstrument.irs import IRS
from jupyterUtils import export
from marketdata import init_marketdata
from marketdata.interestRateIndices import InterestRateIndex
from sa_ccr.sa_ccr import SA_CCR
asdf = 1

The result of Appendix \ref{homogeneity-of-c-for-a-single-trade-portfolio} shows, that inclusion of the MTA results in a local plateau of $C$.

In this section we want to investigate if inclusion of the MTA breaks homogeneity of the SA-CCR EAD function and what can be done to mitigate this effect.

We initialize a 200Bn IRS, a collateral agreement with a 0 threshold and a 1Bn MTA.

In [2]:
irs = IRS(notional = 200000000000,
          timeToSwapStart=ql.Period(2, ql.Days),
          timeToSwapEnd=ql.Period(10, ql.Years),
          index = InterestRateIndex.USDLIBOR3M)
ca = CollateralAgreement(threshold=0,
                         mta=1000000000)
ca.link_sa_ccr_instance(SA_CCR(ca))
ca.add_trades(irs)

ca.set_start_collateral_amount(ca.get_C())

In [3]:
display(Markdown('The starting collateral $C_{t-1}$ is set to %d USD which is also the calculated IM since the VM of this par IRS is %d.' %(ca.get_C(), ca.get_vm_model().get_vm())))

The starting collateral $C_{t-1}$ is set to 9038157077 USD which is also the calculated IM since the VM of this par IRS is 0.

The EAD is:

In [4]:
original_ead = ca.get_sa_ccr_model().get_risk_measure()
original_ead

582881953.4866074

When bumping the notional of the irs by 0.01%, we can see that the collateral of the portfolio does not change due to the MTA.

In [5]:
ca.remove_all_trades()
ca.add_trades(irs.get_bumped_copy(rel_bump_size=0.0001))
ead_bumped_mta = ca.get_sa_ccr_model().get_risk_measure()
print('EAD:  %d' %ead_bumped_mta)
print('C:   %d' %ca.get_C())

EAD:  583024482
C:   9038157077


In [6]:
ead_bumped_mta = ca.get_sa_ccr_model().get_risk_measure()

When temporarily disabling the MTA the resulting EAD and C differ.

In [7]:
ca.mta = 0
ead_bumped_no_mta = ca.get_sa_ccr_model().get_risk_measure()
print('EAD:  %d' %ead_bumped_no_mta)
print('C:   %d' %ca.get_C())

EAD:  582940242
C:   9039060887


Calculating the forward difference in line with equation \todo{reference forward dif eq} with and without consideration of the MTA yields

In [8]:
print('With MTA:    %d' %((ead_bumped_mta-original_ead)/0.0001))
print('Without MTA:  %d' %((ead_bumped_no_mta-original_ead)/0.0001))

With MTA:    1425289375
Without MTA:  582887602


### Impact of the minimum transfer amount on RC

the MTA also impacts RC as displayed in table \ref{tab:Margin in SA-CCR}. Since IM reduces the RC the most relevant case is when the calculated IM is below the threshold.

In an example we try to allocate the EAD of a portfolio consisting of a single 100Mn IRS. The associated collateral agreement has a threshold of 50Mn and a minimum transfer amount of 2Mn.

In [9]:
irs = IRS(notional=100000000,
          timeToSwapStart=ql.Period(2, ql.Days),
          timeToSwapEnd=ql.Period(10, ql.Years),
          index=InterestRateIndex.USDLIBOR3M)

ca = CollateralAgreement(threshold=50000000,
                         mta = 2000000)
ca.link_sa_ccr_instance(SA_CCR(ca))
ca.add_trades(irs)

print('RC:      %d USD' %ca.get_sa_ccr_model().get_rc())
print('RC*1.4:  %d USD' %(ca.get_sa_ccr_model().get_rc()*1.4))
print('PFE:     %d USD' %(ca.get_sa_ccr_model().get_pfe()))
print('PFE*1.4: %d USD' %(ca.get_sa_ccr_model().get_pfe()*1.4))
print('EAD:     %d USD' %ca.get_sa_ccr_model().get_risk_measure())

RC:      2000000 USD
RC*1.4:  2800000 USD
PFE:     1179574 USD
PFE*1.4: 1651404 USD
EAD:     4451404 USD


The EAD is the sum of the RC and the PFE component time the $\alpha$ factor of 1.4.
In all previous examples, the RC has always been floored at 0 since the received IM was higher than the MTA or since the MTA was 0.

Again, Euler allocation is not possible, because the EAD is a sum of the PFE, which is a function of the portfolio notional and the RC which, at least locally, is a constant.
Similar to the issue with threshold described in \ref{exemplary-sa-ccr-allocation-under-consideration-of-an-initial-margin-threshold} one can only allocate without MTA and then allocate the remainder $EAD_{\text{MTA}}-EAD_{\text{no MTA}}$ according to some rule.

Below, we allocate the ead assuming a mta of 0.

In [10]:
ca.mta = 0
eulerAllocator = EulerAllocator(ca)
allocation = eulerAllocator.allocate_ead()
ca.mta = 2000000
allocation[irs]

1651404.724563472

As we can see the result equals $PFE*1.4$ of the entire portfolio but the RC has not been allocated.

The RC is also $>0$, if $TH < IM_{calc} < MTA$. Based on the available marketdata this is e.g. the case for an IRS with a notional of 1130Mn USD.

In [11]:
# In this case it is trivial to allocate the remainder of $EAD_{\text{MTA}} - EAD_{\text{no MTA}}$ to the single trade of the portfolio. Generally, however, no clear, risk-sensitive approach can be identified to allocate the remainder $EAD_{\text{MTA}} - EAD_{\text{no MTA}}$ on the trades of the portfolio after calculating the *'base allocation'* under the assumption of no MTA.

In [12]:
irs2 = IRS(notional=1130000000,
          timeToSwapStart=ql.Period(2, ql.Days),
          timeToSwapEnd=ql.Period(10, ql.Years),
          index=InterestRateIndex.USDLIBOR3M)

ca.remove_all_trades()
ca.add_trades(irs2)

print('RC:        %d USD' %ca.get_sa_ccr_model().get_rc())
print('PFE:     %d USD' %(ca.get_sa_ccr_model().get_pfe()))
print('EAD/1.4: %d USD' %(ca.get_sa_ccr_model().get_risk_measure()/1.4))
print('EAD:     %d USD' %ca.get_sa_ccr_model().get_risk_measure())

RC:        934412 USD
PFE:     13329195 USD
EAD/1.4: 14263607 USD
EAD:     19969050 USD


This observation is discussed further in \todo{Reference appropriate section in Results}.


In [14]:
export('Exemplary SA-CCR allocation under consideration of a minimum transfer amount.ipynb')